#     12.5.3 The Rayleigh Quotient Iteration

With this notebook, we demonstrate how the Inverse Power Method can be accelerated by shifting the matrix, this time by approximating the smallest eigenvalue with the Rayleigh quotient.

<font color=red> Be sure to make a copy!!!! </font>

We start by creating a matrix with known eigenvalues and eigenvectors

How do we do this?  
<ul>
  <li>
    We want a matrix that is not deficient, since otherwise the Rayleigh Quotient Iteration Method may not work. 
  </li>
  <li>
    Hence, $ A = V \Lambda V^{-1} $ for some diagonal matrix $ \Lambda $ and nonsingular matrix $ V $.  The eigenvalues are then on the diagonal of $ \Lambda $ and the eigenvectors are the columns of $ V $.
    </li>
    <li>
    So, let's pick the eigenvalues for the diagonal of $ \Lambda $ and let's pick a random matrix $ V $ (in the hopes that it has linearly independent columns) and then let's see what happens.  
    </li>
    </ul>

<font color=red> Experiment by changing the eigenvalues!  What happens if you make the second entry on the diagonal equal to -4?  Or what if you set 2 to -1? </font>

In [1]:
using LinearAlgebra

Λ = Diagonal([4., 3., 2., 1.])

λ1 = Λ[1, 1]

V = rand(4, 4)

# normalize the columns of V to each have a length of one
for j in 1:4
    V[:, j:j] = V[:, j:j] / sqrt( V[:, j]' * V[:, j])
end

A = V * Λ * inv(V)

println("Λ = ")
Λ

Λ = 


4×4 Diagonal{Float64,Array{Float64,1}}:
 4.0   ⋅    ⋅    ⋅ 
  ⋅   3.0   ⋅    ⋅ 
  ⋅    ⋅   2.0   ⋅ 
  ⋅    ⋅    ⋅   1.0

In [2]:
println("V = ")
V

V = 


4×4 Array{Float64,2}:
 0.449204  0.406523  0.71868    0.28708 
 0.319182  0.231012  0.426402   0.412557
 0.150113  0.564961  0.0612388  0.53101 
 0.820856  0.679847  0.54583    0.682209

In [3]:
println("A = ")
A

A = 


4×4 Array{Float64,2}:
 2.40916   -3.78727  -1.08464  2.54156
 0.60889   -1.35211  -0.97688  1.92655
 1.95404   -4.72681   2.20728  1.09649
 0.947601  -7.34712  -2.17295  6.73566

The idea is as follows:

The eigenvalues of $ A $ are $ \lambda_1, \ldots, \lambda_4 $ with

$$
\vert \lambda_1 \vert > \vert \lambda_2 \vert > \vert \lambda_3 \vert > \vert \lambda_4 \vert > 0
$$

and how fast the iteration converges depends on the ratio 

$$
\left\vert \frac{\lambda_4}{\lambda_3} \right\vert .
$$
Now, if you pick a value, $ \mu $ close to $ \lambda_4 $, and you iterate with $ A - \mu I $ (which is known as shifting the matrix/spectrum by $ \mu $) you can greatly improve the ratio
$$
\left\vert \frac{\lambda_4-\mu}{\lambda_3-\mu} \right\vert .
$$

Generally we don't know $ \lambda_4 $ and hence don't know how to choose $ \mu $.  But we are generating a vector $ x $ that progressively gets closer and closer to an eigenvector.  Thus, we can use the Rayleigh quotient to approximate an eigenvalue.

Here we purposely say "an eigenvalue" since it could be that the first random vector $ x $ is close to an eigenvector associated with another eigenvalue, and then we may converge to a different eigenvalue.

In [21]:
# Pick a random starting vector
x = rand(4)

# Let's start by not shifting, so hopefully we hone in on the smallest eigenvalue
μ = 0.

for i in 1:10
    # We should really compute a factorization of A, but let's be lazy, and compute the inverse
    # explicitly
    Ainv = inv( A - μ * I )
    
    x = Ainv * x 
    
    # normalize x to length one
    x = x / sqrt( x'x )
    
    # Notice we compute the Rayleigh quotient with matrix A, not Ainv.  This is because
    # the eigenvector of A is an eigenvector of Ainv
    μ = x' * A * x

    println("Rayleigh quotient with vector x: $(x'*A*x / (x'x))")
    println("inner product of x with v4     : $(x' * V[:, 4])  \n" )
end

Rayleigh quotient with vector x: 0.4281402869801054
inner product of x with v4     : 0.9911281929874582  

Rayleigh quotient with vector x: 0.9286672113131451
inner product of x with v4     : 0.9989136715004745  

Rayleigh quotient with vector x: 1.0001082759909314
inner product of x with v4     : 0.9999944611308647  

Rayleigh quotient with vector x: 0.9999997344656207
inner product of x with v4     : -0.9999999999999306  

Rayleigh quotient with vector x: 0.9999999999998974
inner product of x with v4     : -1.0  

Rayleigh quotient with vector x: 0.9999999999999993
inner product of x with v4     : -0.9999999999999998  

Rayleigh quotient with vector x: 1.0000000000000009
inner product of x with v4     : -1.0  

Rayleigh quotient with vector x: 1.0000000000000013
inner product of x with v4     : 1.0  

Rayleigh quotient with vector x: 1.0000000000000018
inner product of x with v4     : -0.9999999999999999  

Rayleigh quotient with vector x: 1.0
inner product of x with v4     : 1.0  



In the above, 
 <ul>
 <li>
 The Rayleigh quotient may converge to 1.0 (but it may converge to another eigenvalue!).
 </li>
 <li>
 The inner product of $ x $ and the last column of $ V $, $ v_{n} $, may converge to 1 or -1 since eventually $ x $ may be in the direction of $ v_{n} $ (or in the opposite direction).  But not if we start converging to another eigenvalue... If this happens, try rerunning all the code blocks above to get a different $V$ matrix.
 </li>
 </ul>
